<a href="https://colab.research.google.com/github/William-Metz/NEAT_on_AircraftWings/blob/master/Research_project_work_inprogress_neat_using_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
#Imports for Hosted Runtime



In [ ]:
!git clone https://github.com/William-Metz/OASFORNEURALNETWORKS.git
%cd OASFORNEURALNETWORKS
!pip install -e .
!pip install openmdao
!pip install neat-python

Cloning into 'OASFORNEURALNETWORKS'...
remote: Enumerating objects: 12451, done.
remote: Counting objects: 100% (12451/12451), done.
remote: Compressing objects: 100% (2818/2818), done.
remote: Total 12451 (delta 9462), reused 12444 (delta 9459), pack-reused 0
Receiving objects: 100% (12451/12451), 7.89 MiB | 19.94 MiB/s, done.
Resolving deltas: 100% (9462/9462), done.
/content/OASFORNEURALNETWORKS
Obtaining file:///content/OASFORNEURALNETWORKS
  Running setup.py develop for openaerostruct
    100% |████████████████████████████████| 1.1MB 18.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/8d/fc/c3/ea741c3b2a7d012a123311adf2878da5a914d8b6e58a13bf2e
  Stored in directory: /root/.cache/pip/wheels/b5/b9/ac/f008cca4d5572649ee9c213f8ca8061a1443e77961d44ad4f3
Successfully built openmdao pyDOE2
    100% |████████████████████████████████| 51kB 2.0MB/s 


# BlackBox Script

In [ ]:
"""
This script is an example of a black box setup using OpenAeroStruct.
We first construct and setup a problem, then manually change values in the
prob instance and run analysis at different points. Through this method,
we can manually explore the design space.
Although this example varies angle of attack (alpha), you could vary any input
into the problem, including wing geometry and flight conditions.
"""

from __future__ import division, print_function
import numpy as np

from openaerostruct.geometry.utils import generate_mesh

from openaerostruct.integration.aerostruct_groups import AerostructGeometry, AerostructPoint

from openmdao.api import IndepVarComp, Problem, Group
from openaerostruct.utils.constants import grav_constant


# Create a dictionary to store options about the surface
mesh_dict = {'num_y' : 5,
             'num_x' : 2,
             'wing_type' : 'CRM',
             'symmetry' : True,
             'num_twist_cp' : 5}

mesh, twist_cp = generate_mesh(mesh_dict)

surface = {
            # Wing definition
            'name' : 'wing',        # name of the surface
            'symmetry' : True,     # if true, model one half of wing
                                    # reflected across the plane y = 0
            'S_ref_type' : 'wetted', # how we compute the wing area,
                                     # can be 'wetted' or 'projected'
            'fem_model_type' : 'tube',

            'thickness_cp' : np.array([.1, .2, .3]),

            'twist_cp' : twist_cp,
            'mesh' : mesh,

            # Aerodynamic performance of the lifting surface at
            # an angle of attack of 0 (alpha=0).
            # These CL0 and CD0 values are added to the CL and CD
            # obtained from aerodynamic analysis of the surface to get
            # the total CL and CD.
            # These CL0 and CD0 values do not vary wrt alpha.
            'CL0' : 0.0,            # CL of the surface at alpha=0
            'CD0' : 0.015,            # CD of the surface at alpha=0

            # Airfoil properties for viscous drag calculation
            'k_lam' : 0.05,         # percentage of chord with laminar
                                    # flow, used for viscous drag
            't_over_c_cp' : np.array([0.15]),      # thickness over chord ratio (NACA0015)
            'c_max_t' : .303,       # chordwise location of maximum (NACA0015)
                                    # thickness
            'with_viscous' : True,
            'with_wave' : False,     # if true, compute wave drag

            # Structural values are based on aluminum 7075
            'E' : 70.e9,            # [Pa] Young's modulus of the spar
            'G' : 30.e9,            # [Pa] shear modulus of the spar
            'yield' : 500.e6 / 2.5, # [Pa] yield stress divided by 2.5 for limiting case
            'mrho' : 3.e3,          # [kg/m^3] material density
            'fem_origin' : 0.35,    # normalized chordwise location of the spar
            'wing_weight_ratio' : 2.,
            'struct_weight_relief' : False,    # True to add the weight of the structure to the loads on the structure
            'distributed_fuel_weight' : False,
            # Constraints
            'exact_failure_constraint' : False, # if false, use KS function
            }

# Create the problem and assign the model group
prob = Problem()

# Add problem information as an independent variables component
indep_var_comp = IndepVarComp()
indep_var_comp.add_output('v', val=248.136, units='m/s')
indep_var_comp.add_output('alpha', val=5., units='deg')
indep_var_comp.add_output('Mach_number', val=0.84)
indep_var_comp.add_output('re', val=1.e6, units='1/m')
indep_var_comp.add_output('rho', val=0.38, units='kg/m**3')
indep_var_comp.add_output('CT', val=grav_constant * 17.e-6, units='1/s')
indep_var_comp.add_output('R', val=11.165e6, units='m')
indep_var_comp.add_output('W0', val=0.4 * 3e5,  units='kg')
indep_var_comp.add_output('speed_of_sound', val=295.4, units='m/s')
indep_var_comp.add_output('load_factor', val=1.)
indep_var_comp.add_output('empty_cg', val=np.zeros((3)), units='m')

prob.model.add_subsystem('prob_vars',
     indep_var_comp,
     promotes=['*'])

aerostruct_group = AerostructGeometry(surface=surface)

name = 'wing'

# Add tmp_group to the problem with the name of the surface.
prob.model.add_subsystem(name, aerostruct_group)

point_name = 'AS_point_0'

# Create the aero point group and add it to the model
AS_point = AerostructPoint(surfaces=[surface])

prob.model.add_subsystem(point_name, AS_point,
    promotes_inputs=['v', 'alpha', 'Mach_number', 're', 'rho', 'CT', 'R',
        'W0', 'speed_of_sound', 'empty_cg', 'load_factor'])

com_name = point_name + '.' + name + '_perf'
prob.model.connect(name + '.local_stiff_transformed', point_name + '.coupled.' + name + '.local_stiff_transformed')
prob.model.connect(name + '.nodes', point_name + '.coupled.' + name + '.nodes')

# Connect aerodyamic mesh to coupled group mesh
prob.model.connect(name + '.mesh', point_name + '.coupled.' + name + '.mesh')

# Connect performance calculation variables
prob.model.connect(name + '.radius', com_name + '.radius')
prob.model.connect(name + '.thickness', com_name + '.thickness')
prob.model.connect(name + '.nodes', com_name + '.nodes')
prob.model.connect(name + '.cg_location', point_name + '.' + 'total_perf.' + name + '_cg_location')
prob.model.connect(name + '.structural_mass', point_name + '.' + 'total_perf.' + name + '_structural_mass')
prob.model.connect(name + '.t_over_c', com_name + '.t_over_c')

# Set up the problem
prob.setup()


    # Set the alpha in the problem and run analysi
alphas = np.linspace(-5,5)

# Loopo through each alpha

def testvalue(num):
   alphas = np.linspace(num,num)

# Loopo through each alpha
   for alpha in alphas:
       prob['alpha'] = alpha
       prob.run_model()
       value = prob['AS_point_0.wing_perf.CL']/prob['AS_point_0.wing_perf.CD']
       print( alpha )
       return(value[0])
       break
 


# Optimization Script


In [ ]:
  """
2-input XOR example -- this is most likely the simplest possible example.
"""
from __future__ import print_function
import os
import neat
import sys
from neat import visualize
import csv

# 2-input XOR inputs and expected outputs.
xor_inputs = [(0,0)]
xor_outputs = [   ( 0) ]

def eval_genomes(genomes, config):
    array=[]

    for genome_id, genome in genomes:
      genome.fitness = 4.0
      net = neat.nn.FeedForwardNetwork.create(genome, config)
      for xi, xo in zip(xor_inputs, xor_outputs):
        output = net.activate(xi)
        genome.fitness = (testvalue2(output[0]) -(18.303397)) 
        array.append(output[0])
   
    with open('Simplyfileddata.csv', "a") as file:
      writer = csv.writer(file)
      writer.writerow(array)
    file.close()


     
      
       


def run(config_file):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5))

    # Run for up to 300 generations.
    winner = p.run(eval_genomes, 30)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))

    # Show output of the most fit genome against training data.
    print('\nOutput:')
    winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
    for xi, xo in zip(xor_inputs, xor_outputs):
        output = winner_net.activate(xi)
        print("input {!r}, expected output {!r}, got {!r}".format(xi, xo, output))

    node_names = {-1:'A', -2: 'B', 0:'A XOR B'}
    visualize.draw_net(config, winner, True, node_names=node_names)
    visualize.plot_stats(stats, ylog=False, view=True)
    visualize.plot_species(stats, view=True)

    p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-4')
    p.run(eval_genomes, 30)


if __name__ == '__main__':
    # Determine path to configuration file. This path manipulation is
    # here so that the script will run successfully regardless of the
    # current working directory.
    local_dir = os.path.abspath(os.path.dirname(sys.argv[0]))
    config_path = os.path.join(local_dir, 'Config_simplyified.py')
    run(config_path)


 ****** Running generation 0 ****** 



/content/OASFORNEURALNETWORKS/openaerostruct/structures/compute_nodes.py:53: ComplexWarning: Casting complex values to real discards the imaginary part
  data[:n] = 1 - w
/content/OASFORNEURALNETWORKS/openaerostruct/structures/compute_nodes.py:54: ComplexWarning: Casting complex values to real discards the imaginary part
  data[n:] = w
/content/OASFORNEURALNETWORKS/openaerostruct/transfer/load_transfer.py:57: ComplexWarning: Casting complex values to real discards the imaginary part
  self.fem_origin = np.float(fem_origin)
/usr/local/lib/python3.6/dist-packages/openmdao/core/system.py:1312: ComplexWarning: Casting complex values to real discards the imaginary part
  self._inputs._views[abs_name][:] = abs2meta[abs_name]['value']
/usr/local/lib/python3.6/dist-packages/openmdao/core/system.py:1315: ComplexWarning: Casting complex values to real discards the imaginary part
  self._outputs._views[abs_name][:] = abs2meta[abs_name]['value']
/content/OASFORNEURALNETWORKS/openaerostruct/structu

17.80549271637458
12.548468677756556
18.308538140590016
18.506162150078687
18.79731867037857
18.274874182946576
19.88583536544479
18.3212467156724
18.177811595982202
18.34941041156993
Population's average fitness: -0.40588 stdev: 1.85817
Best fitness: 1.58244 - size: (21, 62) - species 7 - id 7
Average adjusted fitness: 0.729
Mean genetic distance 3.588, standard deviation 0.929
Population of 20 members in 10 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0     2     -0.5    0.716     0
     2    0     2     -5.8    0.000     0
     3    0     2      0.0    0.785     0
     4    0     2      0.2    0.812     0
     5    0     2      0.5    0.852     0
     6    0     2     -0.0    0.780     0
     7    0     2      1.6    1.000     0
     8    0     2      0.0    0.787     0
     9    0     2     -0.1    0.767     0
    10    0     2      0.0    0.791     0
Total extinctions: 0
Generation time: 19.197 sec

 ****** Running generati

In [ ]:
from __future__ import division, print_function

# Ignore the #docs checkpoint comments. They are just used to split up the code for the documentation webpage.
#docs checkpoint 0
import random
import numpy as np
%cd OASFORNEURALNETWORKS

from openaerostruct.geometry.utils import generate_mesh
from openaerostruct.integration.aerostruct_groups import AerostructGeometry, AerostructPoint
from openmdao.api import IndepVarComp, Problem, ScipyOptimizeDriver, SqliteRecorder, ExecComp
from openaerostruct.structures.wingbox_fuel_vol_delta import WingboxFuelVolDelta
from openaerostruct.utils.constants import grav_constant
from random import *
#docs checkpoint 1




def testvalue2(inputval):
  
 
  upper_x = np.array([0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6], dtype = 'complex128')
  lower_x = np.array([0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6], dtype = 'complex128')
  upper_y = np.array([ 0.0447,  0.046,  0.0472,  0.0484,  0.0495,  0.0505,  0.0514,  0.0523,  0.0531,  0.0538, 0.0545,  0.0551,  0.0557, 0.0563,  0.0568, 0.0573,  0.0577,  0.0581,  0.0585,  0.0588,  0.0591,  0.0593,  0.0595,  0.0597,  0.0599,  0.06,    0.0601,  0.0602,  0.0602,  0.0602,  0.0602,  0.0602,  0.0601,  0.06,    0.0599,  0.0598,  0.0596,  0.0594,  0.0592,  0.0589,  0.0586,  0.0583,  0.058,   0.0576,  0.0572,  0.0568,  0.0563,  0.0558,  0.0553,  0.0547,  0.0541], dtype = 'complex128')
  lower_y = np.array([-0.0447, -0.046, -0.0473, -0.0485, -0.0496, -0.0506, -0.0515, -0.0524, -0.0532, -0.054, -0.0547, -0.0554, -0.056, -0.0565, -0.057, -0.0575, -0.0579, -0.0583, -0.0586, -0.0589, -0.0592, -0.0594, -0.0595, -0.0596, -0.0597, -0.0598, -0.0598, -0.0598, -0.0598, -0.0597, -0.0596, -0.0594, -0.0592, -0.0589, -0.0586, -0.0582, -0.0578, -0.0573, -0.0567, -0.0561, -0.0554, -0.0546, -0.0538, -0.0529, -0.0519, -0.0509, -0.0497, -0.0485, -0.0472, -0.0458, -0.0444], dtype = 'complex128')

#docs checkpoint 2

# Create a dictionary to store options about the surface
  mesh_dict = {'num_y' : 15,
             'num_x' : 3,
             'wing_type' : 'uCRM_based',
             'symmetry' : True,
             'chord_cos_spacing' : 0,
             'span_cos_spacing' : 0,
             'num_twist_cp' : 4
             }

  mesh, twist_cp = generate_mesh(mesh_dict)

#docs checkpoint 3

  surf_dict = {
               # Wing definition
            'name' : 'wing',         # give the surface some name
            'symmetry' : True,       # if True, model only one half of the lifting surface
            'S_ref_type' : 'projected', # how we compute the wing area,
                                     # can be 'wetted' or 'projected'
            'mesh' : mesh,

            'fem_model_type' : 'wingbox', # 'wingbox' or 'tube'
            'data_x_upper' : upper_x,
            'data_x_lower' : lower_x,
            'data_y_upper' : upper_y,
            'data_y_lower' : lower_y,

            #docs checkpoint 4

            'twist_cp' : np.array([15., 15., 15., 15.]), # [deg]

            'spar_thickness_cp' : np.array([0.1, 0.1, 0.1, 0.1]), # [m]
            'skin_thickness_cp' : np.array([0.1, 0.1, 0.1, 0.1]),# [m]

            't_over_c_cp' : np.array([0.07, 0.07, 0.07, 0.087]),
            'original_wingbox_airfoil_t_over_c' : 0.12,
            #docs checkpoint 5

            # Aerodynamic deltas.
            # These CL0 and CD0 values are added to the CL and CD
            # obtained from aerodynamic analysis of the surface to get
            # the total CL and CD.
            # These CL0 and CD0 values do not vary wrt alpha.
            # They can be used to account for things that are not included, such as contributions from the fuselage, camber, etc.
            'CL0' : 0.0,            # CL delta
            'CD0' : 0.000,         # CD delta

            'with_viscous' : True,  # if true, compute viscous drag
            'with_wave' : True,     # if true, compute wave drag

            # Airfoil properties for viscous drag calculation
            'k_lam' : 0.05,         # fraction of chord with laminar
                                    # flow, used for viscous drag
            'c_max_t' : .38,       # chordwise location of maximum thickness

            #docs checkpoint 6

            # Structural values are based on aluminum 7075
            'E' : 73.1e9,              # [Pa] Young's modulus
            'G' : (73.1e9/2/1.33),     # [Pa] shear modulus (calculated using E and the Poisson's ratio here)
            'yield' : (420.e6 / 1.5),  # [Pa] allowable yield stress
            'mrho' : 2.78e3,           # [kg/m^3] material density
            'strength_factor_for_upper_skin' : 1.0, # the yield stress is multiplied by this factor for the upper skin

            'wing_weight_ratio' : 1.25,
            'exact_failure_constraint' : False, # if false, use KS function

            #docs checkpoint 7

            'struct_weight_relief' : True,
            'distributed_fuel_weight' : True,

            #docs checkpoint 8

            'fuel_density' : 803.,      # [kg/m^3] fuel density (only needed if the fuel-in-wing volume constraint is used)
            'Wf_reserve' :0.01,       # [kg] reserve fuel mass

           
            }

  surfaces = [surf_dict]

#docs checkpoint 9

# Create the problem and assign the model group
  prob = Problem()

# Add problem information as an independent variables component
  indep_var_comp = IndepVarComp()
  indep_var_comp.add_output('Mach_number', val=np.array([0.85, 0.64]))
  indep_var_comp.add_output('v', val=np.array([.85 * 295.07, .64 * 340.294]), units='m/s')
  indep_var_comp.add_output('re',val=np.array([0.348*295.07*.85*1./(1.43*1e-5), \
                          1.225*340.294*.64*1./(1.81206*1e-5)]),  units='1/m')
  indep_var_comp.add_output('rho', val=np.array([0.348, 1.225]), units='kg/m**3')
  indep_var_comp.add_output('speed_of_sound', val= np.array([295.07, 340.294]), units='m/s')

#docs checkpoint 10

  indep_var_comp.add_output('CT', val=0.53/3600, units='1/s')
  indep_var_comp.add_output('R', val=14.307e6, units='m')
  indep_var_comp.add_output('W0', val=148000 + surf_dict['Wf_reserve'],  units='kg')

#docs checkpoint 11

  indep_var_comp.add_output('load_factor', val=np.array([1., 2.5]))
  indep_var_comp.add_output('alpha', val=inputval, units='deg')
  indep_var_comp.add_output('alpha_maneuver', val=0., units='deg')

  indep_var_comp.add_output('empty_cg', val=np.zeros((3)), units='m')

#docs checkpoint 12

  indep_var_comp.add_output('fuel_mass', val=0.0001, units='kg')

  prob.model.add_subsystem('prob_vars',
      indep_var_comp,
       promotes=['*'])
  
#docs checkpoint 13

# Loop over each surface in the surfaces list
  for surface in surfaces:

    # Get the surface name and create a group to contain components
    # only for this surface
      name = surface['name']

      aerostruct_group = AerostructGeometry(surface=surface)

    # Add groups to the problem with the name of the surface.
      prob.model.add_subsystem(name, aerostruct_group)

#docs checkpoint 14

# Loop through and add a certain number of aerostruct points
  for i in range(2):

      point_name = 'AS_point_{}'.format(i)
    # Connect the parameters within the model for each aero point

    # Create the aerostruct point group and add it to the model
      AS_point = AerostructPoint(surfaces=surfaces, internally_connect_fuelburn=False)

      prob.model.add_subsystem(point_name, AS_point)

    #docs checkpoint 15

    # Connect flow properties to the analysis point
      prob.model.connect('v', point_name + '.v', src_indices=[i])
      prob.model.connect('Mach_number', point_name + '.Mach_number', src_indices=[i])
      prob.model.connect('re', point_name + '.re', src_indices=[i])
      prob.model.connect('rho', point_name + '.rho', src_indices=[i])
      prob.model.connect('CT', point_name + '.CT')
      prob.model.connect('R', point_name + '.R')
      prob.model.connect('W0', point_name + '.W0')
      prob.model.connect('speed_of_sound', point_name + '.speed_of_sound', src_indices=[i])
      prob.model.connect('empty_cg', point_name + '.empty_cg')
      prob.model.connect('load_factor', point_name + '.load_factor', src_indices=[i])
      prob.model.connect('fuel_mass', point_name + '.total_perf.L_equals_W.fuelburn')
      prob.model.connect('fuel_mass', point_name + '.total_perf.CG.fuelburn')

    #docs checkpoint 16

      for surface in surfaces:
        name = surface['name']

        if surf_dict['distributed_fuel_weight']:
            prob.model.connect('load_factor', point_name + '.coupled.load_factor', src_indices=[i])

        com_name = point_name + '.' + name + '_perf.'
        prob.model.connect(name + '.local_stiff_transformed', point_name + '.coupled.' + name + '.local_stiff_transformed')
        prob.model.connect(name + '.nodes', point_name + '.coupled.' + name + '.nodes')

        # Connect aerodyamic mesh to coupled group mesh
        prob.model.connect(name + '.mesh', point_name + '.coupled.' + name + '.mesh')
        if surf_dict['struct_weight_relief']:
            prob.model.connect(name + '.element_mass', point_name + '.coupled.' + name + '.element_mass')

        # Connect performance calculation variables
        prob.model.connect(name + '.nodes', com_name + 'nodes')
        prob.model.connect(name + '.cg_location', point_name + '.' + 'total_perf.' + name + '_cg_location')
        prob.model.connect(name + '.structural_mass', point_name + '.' + 'total_perf.' + name + '_structural_mass')

        # Connect wingbox properties to von Mises stress calcs
        prob.model.connect(name + '.Qz', com_name + 'Qz')
        prob.model.connect(name + '.J', com_name + 'J')
        prob.model.connect(name + '.A_enc', com_name + 'A_enc')
        prob.model.connect(name + '.htop', com_name + 'htop')
        prob.model.connect(name + '.hbottom', com_name + 'hbottom')
        prob.model.connect(name + '.hfront', com_name + 'hfront')
        prob.model.connect(name + '.hrear', com_name + 'hrear')

        prob.model.connect(name + '.spar_thickness', com_name + 'spar_thickness')
        prob.model.connect(name + '.t_over_c', com_name + 't_over_c')


      


#docs checkpoint 17

  prob.model.connect('alpha', 'AS_point_0' + '.alpha')
  prob.model.connect('alpha_maneuver', 'AS_point_1' + '.alpha')

#docs checkpoint 18

# Here we add the fuel volume constraint componenet to the model
  prob.model.add_subsystem('fuel_vol_delta', WingboxFuelVolDelta(surface=surface))
  prob.model.connect('wing.struct_setup.fuel_vols', 'fuel_vol_delta.fuel_vols')
  prob.model.connect('AS_point_0.fuelburn', 'fuel_vol_delta.fuelburn')

  if surf_dict['distributed_fuel_weight']:
    prob.model.connect('wing.struct_setup.fuel_vols', 'AS_point_0.coupled.wing.struct_states.fuel_vols')
    prob.model.connect('fuel_mass', 'AS_point_0.coupled.wing.struct_states.fuel_mass')

    prob.model.connect('wing.struct_setup.fuel_vols', 'AS_point_1.coupled.wing.struct_states.fuel_vols')
    prob.model.connect('fuel_mass', 'AS_point_1.coupled.wing.struct_states.fuel_mass')

#docs checkpoint 19
  

  
#docs checkpoint 20

  
  prob.model.add_objective('AS_point_0.fuelburn', scaler=1e-5)

  prob.model.add_design_var('wing.twist_cp', lower=-15., upper=15., scaler=0.1)
  prob.model.add_design_var('wing.spar_thickness_cp', lower=0.003, upper=0.1, scaler=1e2)
  prob.model.add_design_var('wing.skin_thickness_cp', lower=0.003, upper=0.1, scaler=1e2)
  prob.model.add_design_var('wing.geometry.t_over_c_cp', lower=0.07, upper=0.2, scaler=10.)
  prob.model.add_design_var('alpha_maneuver', lower=-15., upper=15)
  prob.model.add_design_var('alpha', lower =inputval, upper = inputval)

#docs checkpoint 21

  prob.model.add_constraint('AS_point_0.CL', equals=0.5)

#docs checkpoint 22

  prob.model.add_constraint('AS_point_1.L_equals_W', equals=0.)
  

#docs checkpoint 23


#docs checkpoint 24


#docs checkpoint 25



#docs checkpoint 26

  recorder = SqliteRecorder("fillername.db")
  prob.driver.add_recorder(recorder)

# We could also just use prob.driver.recording_options['includes']=['*'] here, but for large meshes the database file becomes extremely large. So we just select the variables we need.
  prob.driver.recording_options['includes'] =[
    'alpha', 'rho', 'v', 'cg',
    'AS_point_1.cg', 'AS_point_0.cg',
    'AS_point_0.coupled.wing_loads.loads',
    'AS_point_1.coupled.wing_loads.loads',
    'AS_point_0.coupled.wing.normals',
    'AS_point_1.coupled.wing.normals',
    'AS_point_0.coupled.wing.widths',
    'AS_point_1.coupled.wing.widths',
    'AS_point_0.coupled.aero_states.wing_sec_forces',
    'AS_point_1.coupled.aero_states.wing_sec_forces',
    'AS_point_0.wing_perf.CL1',
    'AS_point_1.wing_perf.CL1',
    'AS_point_0.coupled.wing.S_ref',
    'AS_point_1.coupled.wing.S_ref',
    'wing.geometry.twist',
    'wing.mesh',
    'wing.skin_thickness',
    'wing.spar_thickness',
    'wing.t_over_c',
    'AS_point_0.wing_perf.vonmises',
    'AS_point_1.wing_perf.vonmises',
    'AS_point_0.coupled.wing.def_mesh',
    'AS_point_1.coupled.wing.def_mesh',
    ]

  prob.driver.recording_options['record_objectives'] = True
  prob.driver.recording_options['record_constraints'] = True
  prob.driver.recording_options['record_desvars'] = True
  prob.driver.recording_options['record_inputs'] = True

#docs checkpoint 27

# Set up the problem
  prob.setup()

# from openmdao.api import view_model
# view_model(prob)

# prob.check_partials(form='central', compact_print=True)

# Loopo through each alpha
    # Set the alpha in the problem and run analysis
   
  prob.run_model()

      
     
  value = (prob['AS_point_0.wing_perf.CL']/prob['AS_point_0.wing_perf.CD'])
  print(value[0])
  
  return(value[0])

print(testvalue2(0))

/content/OASFORNEURALNETWORKS/OASFORNEURALNETWORKS


/content/OASFORNEURALNETWORKS/openaerostruct/structures/compute_nodes.py:53: ComplexWarning: Casting complex values to real discards the imaginary part
  data[:n] = 1 - w
/content/OASFORNEURALNETWORKS/openaerostruct/structures/compute_nodes.py:54: ComplexWarning: Casting complex values to real discards the imaginary part
  data[n:] = w
/content/OASFORNEURALNETWORKS/openaerostruct/transfer/load_transfer.py:57: ComplexWarning: Casting complex values to real discards the imaginary part
  self.fem_origin = np.float(fem_origin)
/usr/local/lib/python3.6/dist-packages/openmdao/core/system.py:1312: ComplexWarning: Casting complex values to real discards the imaginary part
  self._inputs._views[abs_name][:] = abs2meta[abs_name]['value']
/usr/local/lib/python3.6/dist-packages/openmdao/core/system.py:1315: ComplexWarning: Casting complex values to real discards the imaginary part
  self._outputs._views[abs_name][:] = abs2meta[abs_name]['value']


18.303397516165234
18.303397516165234


/content/OASFORNEURALNETWORKS/openaerostruct/structures/wingbox_geometry.py:82: ComplexWarning: Casting complex values to real discards the imaginary part
  fem_chords[ielem] = fem_chords[ielem] * cos_theta_fe_sweep
/usr/local/lib/python3.6/dist-packages/openmdao/vectors/vector.py:324: ComplexWarning: Casting complex values to real discards the imaginary part
  self._views[abs_name][slc] = value


#Temporary code to get initial fitness for new inputs for Optimization 

In [ ]:
""""


This script is an example of a black box setup using OpenAeroStruct.
We first construct and setup a problem, then manually change values in the
prob instance and run analysis at different points. Through this method,
we can manually explore the design space.
Although this example varies angle of attack (alpha), you could vary any input
into the problem, including wing geometry and flight conditions.


from __future__ import division, print_function
import numpy as np

from openaerostruct.geometry.utils import generate_mesh

from openaerostruct.integration.aerostruct_groups import AerostructGeometry, AerostructPoint

from openmdao.api import IndepVarComp, Problem, Group
from openaerostruct.utils.constants import grav_constant

# Create a dictionary to store options about the surface
mesh_dict = {'num_y' : 5,
             'num_x' : 2,
             'wing_type' : 'CRM',
             'symmetry' : True,
             'num_twist_cp' : 5}

mesh, twist_cp = generate_mesh(mesh_dict)

surface = {
            # Wing definition
            'name' : 'wing',        # name of the surface
            'symmetry' : True,     # if true, model one half of wing
                                    # reflected across the plane y = 0
            'S_ref_type' : 'wetted', # how we compute the wing area,
                                     # can be 'wetted' or 'projected'
            'fem_model_type' : 'tube',

            'thickness_cp' : np.array([.1, .2, .3]),

            'twist_cp' : twist_cp,
            'mesh' : mesh,

            # Aerodynamic performance of the lifting surface at
            # an angle of attack of 0 (alpha=0).
            # These CL0 and CD0 values are added to the CL and CD
            # obtained from aerodynamic analysis of the surface to get
            # the total CL and CD.
            # These CL0 and CD0 values do not vary wrt alpha.
            'CL0' : 0.0,            # CL of the surface at alpha=0
            'CD0' : 0.015,            # CD of the surface at alpha=0

            # Airfoil properties for viscous drag calculation
            'k_lam' : 0.05,         # percentage of chord with laminar
                                    # flow, used for viscous drag
            't_over_c_cp' : np.array([0.15]),      # thickness over chord ratio (NACA0015)
            'c_max_t' : .303,       # chordwise location of maximum (NACA0015)
                                    # thickness
            'with_viscous' : True,
            'with_wave' : False,     # if true, compute wave drag

            # Structural values are based on aluminum 7075
            'E' : 70.e9,            # [Pa] Young's modulus of the spar
            'G' : 30.e9,            # [Pa] shear modulus of the spar
            'yield' : 500.e6 / 2.5, # [Pa] yield stress divided by 2.5 for limiting case
            'mrho' : 3.e3,          # [kg/m^3] material density
            'fem_origin' : 0.35,    # normalized chordwise location of the spar
            'wing_weight_ratio' : 2.,
            'struct_weight_relief' : False,    # True to add the weight of the structure to the loads on the structure
            'distributed_fuel_weight' : False,
            # Constraints
            'exact_failure_constraint' : False, # if false, use KS function
            }

# Create the problem and assign the model group
prob = Problem()

# Add problem information as an independent variables component
indep_var_comp = IndepVarComp()
indep_var_comp.add_output('v', val=248.136, units='m/s')
indep_var_comp.add_output('alpha', val=5., units='deg')
indep_var_comp.add_output('Mach_number', val=0.84)
indep_var_comp.add_output('re', val=1.e6, units='1/m')
indep_var_comp.add_output('rho', val=0.38, units='kg/m**3')
indep_var_comp.add_output('CT', val=grav_constant * 17.e-6, units='1/s')
indep_var_comp.add_output('R', val=11.165e6, units='m')
indep_var_comp.add_output('W0', val=0.4 * 3e5,  units='kg')
indep_var_comp.add_output('speed_of_sound', val=295.4, units='m/s')
indep_var_comp.add_output('load_factor', val=1.)
indep_var_comp.add_output('empty_cg', val=np.zeros((3)), units='m')

prob.model.add_subsystem('prob_vars',
     indep_var_comp,
     promotes=['*'])

aerostruct_group = AerostructGeometry(surface=surface)

name = 'wing'

# Add tmp_group to the problem with the name of the surface.
prob.model.add_subsystem(name, aerostruct_group)

point_name = 'AS_point_0'

# Create the aero point group and add it to the model
AS_point = AerostructPoint(surfaces=[surface])

prob.model.add_subsystem(point_name, AS_point,
    promotes_inputs=['v', 'alpha', 'Mach_number', 're', 'rho', 'CT', 'R',
        'W0', 'speed_of_sound', 'empty_cg', 'load_factor'])

com_name = point_name + '.' + name + '_perf'
prob.model.connect(name + '.local_stiff_transformed', point_name + '.coupled.' + name + '.local_stiff_transformed')
prob.model.connect(name + '.nodes', point_name + '.coupled.' + name + '.nodes')

# Connect aerodyamic mesh to coupled group mesh
prob.model.connect(name + '.mesh', point_name + '.coupled.' + name + '.mesh')

# Connect performance calculation variables
prob.model.connect(name + '.radius', com_name + '.radius')
prob.model.connect(name + '.thickness', com_name + '.thickness')
prob.model.connect(name + '.nodes', com_name + '.nodes')
prob.model.connect(name + '.cg_location', point_name + '.' + 'total_perf.' + name + '_cg_location')
prob.model.connect(name + '.structural_mass', point_name + '.' + 'total_perf.' + name + '_structural_mass')
prob.model.connect(name + '.t_over_c', com_name + '.t_over_c')

# Set up the problem
prob.setup()


    # Set the alpha in the problem and run analysi
alphas = np.linspace(200,200)

# Loopo through each alpha
for alpha in alphas:
  prob['alpha'] = alpha
  prob.run_model()
  value = prob['AS_point_0.wing_perf.CL']/prob['AS_point_0.wing_perf.CD']
  print ("Fitness:", value[0])
  break
  
  
  
 """"


In [ ]:
print(testvalue(10.748916246169703))

16.076069479527504  angle:  10.748916246169703
16.076069479527504
